In [1]:
# Import libraries
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import requests
import matplotlib.pyplot as plt
import json
import re
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
# # Read csv and create dataframe
# from google.colab import files
# uploaded = files.upload()

In [3]:
import io
salaries_df = pd.read_csv('Resources/salaries_cleaned.csv')

In [4]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
salaries_df.head()

,Unnamed: 0,timestamp,date,company,level,title,totalyearlycompensation,location,latitude,longitude,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,Race,Education
0,0,2017-06-07 11:33:00,2017-06-07 00:00:00,Oracle,L3,Product Manager,127000,"Redwood City, CA",37.486324,-122.232523,1.5,1.5,107000,20000,10000,Unknown,Unknown,Unknown
1,1,2017-06-10 17:11:00,2017-06-10 00:00:00,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",37.779026,-122.419906,5.0,3.0,0,0,0,Unknown,Unknown,Unknown
2,2,2017-06-11 14:53:00,2017-06-11 00:00:00,Amazon,L7,Product Manager,310000,"Seattle, WA",47.603832,-122.330062,8.0,0.0,155000,0,0,Unknown,Unknown,Unknown
3,3,2017-06-17 00:23:00,2017-06-17 00:00:00,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",37.368830,-122.036349,7.0,5.0,157000,180000,35000,Unknown,Unknown,Unknown
4,4,2017-06-20 10:58:00,2017-06-20 00:00:00,Microsoft,60,Software Engineer,157000,"Mountain View, CA",37.389389,-122.083210,5.0,3.0,0,0,0,Unknown,Unknown,Unknown


In [6]:
salaries_df.columns

Index(['Unnamed: 0', 'timestamp', 'date', 'company', 'level', 'title',
       'totalyearlycompensation', 'location', 'latitude', 'longitude',
       'yearsofexperience', 'yearsatcompany', 'basesalary', 'stockgrantvalue',
       'bonus', 'gender', 'Race', 'Education'],
      dtype='object')

In [32]:
 salaries_df.groupby('title').describe()['totalyearlycompensation']

,count,mean,std,min,25%,50%,75%,max
title,,,,,,,,
Business Analyst,885.0,129727.683616,67271.511823,10000.0,87000.0,116000.0,160000.0,650000.0
Data Scientist,2578.0,203656.710628,109504.643967,10000.0,135000.0,185000.0,250000.0,1000000.0
Hardware Engineer,2200.0,213655.000000,108318.822574,15000.0,140000.0,193500.0,260000.0,1150000.0
Human Resources,364.0,178711.538462,101163.325811,10000.0,104750.0,164500.0,230000.0,525000.0
Management Consultant,976.0,162795.081967,98473.607118,12000.0,100000.0,149000.0,200000.0,1600000.0
Marketing,710.0,198971.830986,115845.414218,13000.0,129250.0,175000.0,240000.0,1470000.0
Mechanical Engineer,490.0,158442.857143,87963.611411,10000.0,100000.0,133000.0,200000.0,615000.0
Product Designer,1516.0,207637.203166,109739.200170,10000.0,138000.0,187500.0,251000.0,950000.0
Product Manager,4673.0,257813.396105,182258.214578,10000.0,163000.0,220000.0,305000.0,4980000.0


In [7]:
salaries_df['company'].value_counts()

Amazon                      8126
Microsoft                   5216
Google                      4330
Facebook                    2990
Apple                       2028
                            ... 
Samsung research America       1
Bny Mellon                     1
yelp                           1
Bloomberg lp                   1
tableau software               1
Name: company, Length: 1631, dtype: int64

In [8]:
val=salaries_df['company'].value_counts()
print(val[val>200])

Amazon               8126
Microsoft            5216
Google               4330
Facebook             2990
Apple                2028
Oracle               1128
Salesforce           1056
Intel                 949
Cisco                 907
IBM                   907
Uber                  880
Capital One           778
LinkedIn              701
VMware                657
Qualcomm              565
JPMorgan Chase        541
Bloomberg             537
Goldman Sachs         469
Intuit                449
PayPal                426
Deloitte              401
Twitter               395
Adobe                 383
Walmart Labs          381
Nvidia                357
Accenture             355
Expedia               352
Wayfair               338
eBay                  338
SAP                   330
Lyft                  325
Netflix               268
Workday               262
Dropbox               254
Shopify               234
Airbnb                229
Dell Technologies     228
Atlassian             219
Visa        

In [9]:
#top 20 companies
comapanies_to_replace=[]
company=salaries_df['company'].value_counts()
company=pd.DataFrame(company)
company.head(20)

,company
Amazon,8126
Microsoft,5216
Google,4330
Facebook,2990
Apple,2028
Oracle,1128
Salesforce,1056
Intel,949
Cisco,907
IBM,907


In [10]:
comapanies_to_replace.append(company[company.company<430].index)

In [11]:
#Replaced with other for all companies besides the top 20
for x in comapanies_to_replace:
    salaries_df['company'] = salaries_df['company'].replace(x,"Other")
    
# Check to make sure binning was successful
salaries_df['company'].value_counts()

Other             29423
Amazon             8126
Microsoft          5216
Google             4330
Facebook           2990
Apple              2028
Oracle             1128
Salesforce         1056
Intel               949
Cisco               907
IBM                 907
Uber                880
Capital One         778
LinkedIn            701
VMware              657
Qualcomm            565
JPMorgan Chase      541
Bloomberg           537
Goldman Sachs       469
Intuit              449
Name: company, dtype: int64

In [12]:
reduced_df = salaries_df[[ 'title', 'totalyearlycompensation',
       'yearsofexperience', 'yearsatcompany']]

In [13]:
reduced_df


,title,totalyearlycompensation,yearsofexperience,yearsatcompany
0,Product Manager,127000,1.5,1.5
1,Software Engineer,100000,5.0,3.0
2,Product Manager,310000,8.0,0.0
3,Software Engineering Manager,372000,7.0,5.0
4,Software Engineer,157000,5.0,3.0
...,...,...,...,...
62637,Software Engineer,327000,10.0,1.0
62638,Software Engineer,237000,2.0,2.0
62639,Software Engineer,220000,14.0,12.0
62640,Software Engineer,280000,8.0,4.0


In [14]:
dummies_df = pd.get_dummies(reduced_df)
dummies_df.head()

,totalyearlycompensation,yearsofexperience,yearsatcompany,title_Business Analyst,title_Data Scientist,title_Hardware Engineer,title_Human Resources,title_Management Consultant,title_Marketing,title_Mechanical Engineer,title_Product Designer,title_Product Manager,title_Recruiter,title_Sales,title_Software Engineer,title_Software Engineering Manager,title_Solution Architect,title_Technical Program Manager
0,127000,1.5,1.5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,100000,5.0,3.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,310000,8.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,372000,7.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,157000,5.0,3.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [15]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(columns=["totalyearlycompensation"]).values
y = dummies_df["totalyearlycompensation"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
# # Split our preprocessed data into our features and target arrays
# X = reduced_df.drop(columns=["totalyearlycompensation"]).values
# y = reduced_df["totalyearlycompensation"].values

# # Split the preprocessed data into a training and testing dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [17]:
X.shape

(62642, 17)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:

#https://www.simplilearn.com/tutorials/machine-learning-tutorial/stock-price-prediction-using-machine-learning
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = (len(dummies_df.columns)-1)
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.LSTM(units=80, return_sequences=True, input_shape=(X_train.shape[1],1)))

# Second hidden layer
nn.add(tf.keras.layers.LSTM(units=30, return_sequences=True))

# Output layer
nn.add(tf.keras.layers.Dense(units=1))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 17, 80)            26240     
                                                                 
 lstm_1 (LSTM)               (None, 17, 30)            13320     
                                                                 
 dense (Dense)               (None, 17, 1)             31        
                                                                 
Total params: 39,591
Trainable params: 39,591
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")